In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
print("loaded data3")

import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
loaded data3


In [2]:
# abstracting functions

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def conv2d(W, x):
    return tf.nn.conv2d(input=x, filter=W, strides=[1, 1, 1, 1], padding="SAME")
def max_pool_2x2(x):
    return tf.nn.max_pool(value=x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope("summaries"):
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean)
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar("stddev", stddev)
        tf.summary.histogram("histogram", var)

def fc_nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """ creates a fully connected layer"""
    with tf.name_scope(layer_name):
        with tf.name_scope("weights"):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
            
        with tf.name_scope("biases"):
            biases  = bias_variable([output_dim])
            variable_summaries(biases)
            
        with tf.name_scope("Wx_plus_b"):
            preactivate = tf.matmul(input_tensor, weights) + biases
            #tf.summary.histogram("pre_activations", preactivate)
            
        activations = act(preactivate, name="activation")
        #tf.summary.histogram("activations", activations)
        return activations

def conv_nn_layer(input_tensor, patch_size, input_depth, output_depth, layer_name, act=tf.nn.relu):
    """ Create a convolutional layer and performs max pooling, halfing spatial dim"""
    with tf.name_scope(layer_name):
        with tf.name_scope("weights"):
            weights = weight_variable([patch_size, patch_size, input_depth, output_depth])
            variable_summaries(weights)
            
        with tf.name_scope("biases"):
            biases = bias_variable([output_depth])
            variable_summaries(biases)
            
        with tf.name_scope("conv"):
            conv = conv2d(weights, input_tensor)
            #tf.summary.histogram("pre_activations", conv)
            
    activations = act(conv, name="activation")
    #tf.summary.histogram("activations", activations)
    return tf.nn.max_pool(activations, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
        
        


In [3]:
# defining the computation graph

num_steps = 2000

with tf.name_scope("input"):
    x      = tf.placeholder(tf.float32, shape=[None, 784], name="x-input")
    y_true = tf.placeholder(tf.float32, shape=[None, 10],  name="y-input")

with tf.name_scope("input_reshape"):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image("input", image_shaped_input, 10)


# 2 convolutional layers with max pooling
conv1 = conv_nn_layer(image_shaped_input, 5, 1, 30, "conv1")

conv2 = conv_nn_layer(conv1, 5, 30, 60, "conv2")

# reshape to let input fit fully connected layer
shape = conv2.get_shape().as_list()
dim = np.prod(shape[1:])
reshape = tf.reshape(conv2, [-1, dim])

    
fc1 = fc_nn_layer(reshape, dim, 300, "fc1")

with tf.name_scope("dropout1"):
    keep_prob1 = tf.placeholder(tf.float32)
    dropped1 = tf.nn.dropout(fc1, keep_prob1, name="dropout1")
    
fc2 = fc_nn_layer(dropped1, 300, 100, "fc2")

with tf.name_scope("dropout2"):
    keep_prob2 = tf.placeholder(tf.float32)
    dropped2 = tf.nn.dropout(fc2, keep_prob2, name="dropout2")
    
y_pred = fc_nn_layer(dropped2, 100, 10, "fc3", act=tf.identity)



with tf.name_scope("loss_function"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    loss = tf.reduce_mean(cross_entropy)
    loss += 1e-5 * ( tf.nn.l2_loss(conv1) 
                   + tf.nn.l2_loss(conv2)
                   + tf.nn.l2_loss(fc1)
                   + tf.nn.l2_loss(fc2))
tf.summary.scalar("loss", loss)

with tf.name_scope("learning_rate"):
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.01, global_step, 500, 0.7)
tf.summary.scalar("learning_rate", learning_rate)

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss, global_step=global_step)

with tf.name_scope("accuracy"):
    with tf.name_scope("correct_prediciton"):
        correct_prediction = tf.equal(tf.argmax(y_true, 1), tf.argmax(y_pred, 1))
    with tf.name_scope("accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar("accuracy", accuracy)


<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [4]:
# Merge all summaries and write them to tmp/tensorflow/mnist/logs/mnist_with_summaries
path = "C:/tmp/tensorflow/mnist/logs/mnist_with_summaries"
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(path + "/train", sess.graph)
test_writer  = tf.summary.FileWriter(path + "/test",  sess.graph)
tf.global_variables_initializer().run()

In [5]:
for i in range(num_steps):
    if i % 100 == 0: #record summaries and test_set accuracy
        feed_dict = {x: mnist.test.images, y_true: mnist.test.labels, keep_prob1: 1.0, keep_prob2: 1.0}
        summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict)
        test_writer.add_summary(summary, i)
        print('Accuracy at step %s: %s' % (i, acc))
    
    x_batch, y_batch = mnist.train.next_batch(100)
    feed_dict = {x: x_batch, y_true: y_batch, keep_prob1: 0.5, keep_prob2: 0.5}
    summary, _ = sess.run([merged, train_step], feed_dict=feed_dict)
    train_writer.add_summary(summary, i)
        
train_writer.close()
test_writer.close()

Accuracy at step 0: 0.0472
Accuracy at step 100: 0.9611
Accuracy at step 200: 0.9754
Accuracy at step 300: 0.9759
Accuracy at step 400: 0.9768
Accuracy at step 500: 0.9805
Accuracy at step 600: 0.9825
Accuracy at step 700: 0.9818
Accuracy at step 800: 0.9798
Accuracy at step 900: 0.983
Accuracy at step 1000: 0.9839
Accuracy at step 1100: 0.9854
Accuracy at step 1200: 0.988
Accuracy at step 1300: 0.9894
Accuracy at step 1400: 0.9863
Accuracy at step 1500: 0.9882
Accuracy at step 1600: 0.9872
Accuracy at step 1700: 0.9882
Accuracy at step 1800: 0.9883
Accuracy at step 1900: 0.9881
